## Calcular Distancias de Escuelas a establecimientos que venden alcohol mas cercanos

# Chequen los PATHS 

Primero se importan los scripts y scipy

In [2]:
from scripts.filter import filtrar_dataframe
from scripts.distance import find_nearest_n_places, find_nearest_within_radius
from scipy.spatial import KDTree
import pandas as pd


# Seccion de PATHS

In [3]:
# Lista de rutas de archivos Negocios DENUE
paths_negocios_denue = [
    # AQUI AGREGUEN LOS PATHS DE SUS CSVs
    '/home/jay/repos/AI/feminicidios/denue_00_46111_csv/conjunto_de_datos/denue_inegi_46111_.csv',
    '/home/jay/repos/AI/feminicidios/denue_00_46112-46311_csv/conjunto_de_datos/denue_inegi_46112-46311_.csv',
    '/home/jay/repos/AI/feminicidios/denue_00_46321-46531_csv/conjunto_de_datos/denue_inegi_46321-46531_.csv',
    '/home/jay/repos/AI/feminicidios/denue_00_46591-46911_csv/conjunto_de_datos/denue_inegi_46591-46911_.csv',
    
    # Sin querer puse el PATH de las escuelas, xd que bueno que despues de get_alcohol() el DF estaba vacio
    # Lo deje para debuggear
    # '/home/jay/repos/AI/feminicidios/denue_00_61_csv/conjunto_de_datos/denue_inegi_61_.csv'
]

# Ruta de archivo Escuelas DENUE
file_path_escuelas = '/home/jay/repos/AI/feminicidios/denue_00_61_csv/conjunto_de_datos/denue_inegi_61_.csv'

# Ruta de archivo salud DENUE
file_path_salud = '/home/jay/repos/GeoStats/denue/denue_00_62_csv/conjunto_de_datos/denue_inegi_62_.csv'

# Ruta de archivo parques DENUE
file_path_parques = '/home/jay/repos/GeoStats/denue/denue_00_71_csv/conjunto_de_datos/denue_inegi_71_.csv'

# Ruta de archivo mayoreo DENUE
file_path_mayoreo = '/home/jay/repos/GeoStats/denue/denue_00_43_csv/conjunto_de_datos/denue_inegi_43_.csv'

if True:
    # Lista de rutas de archivos Negocios DENUE
    paths_negocios_denue = [
        # AQUI AGREGUEN LOS PATHS DE SUS CSVs
        '/Users/jay/GeoStats/denue/denue_00_46111_csv/conjunto_de_datos/denue_inegi_46111_.csv',
        '/Users/jay/GeoStats/denue/denue_00_46112-46311_csv/conjunto_de_datos/denue_inegi_46112-46311_.csv',
        '/Users/jay/GeoStats/denue/denue_00_46321-46531_csv/conjunto_de_datos/denue_inegi_46321-46531_.csv',
        '/Users/jay/GeoStats/denue/denue_00_46591-46911_csv/conjunto_de_datos/denue_inegi_46591-46911_.csv',
        
        # Sin querer puse el PATH de las escuelas, xd que bueno que despues de get_alcohol() el DF estaba vacio
        # Lo deje para debuggear
        # '/home/jay/repos/AI/feminicidios/denue_00_61_csv/conjunto_de_datos/denue_inegi_61_.csv'
    ]

    # Ruta de archivo Escuelas DENUE
    file_path_escuelas = '/Users/jay/GeoStats/denue/denue_00_61_csv/conjunto_de_datos/denue_inegi_61_.csv'

    # Ruta de archivo salud DENUE
    file_path_salud = '/home/jay/repos/GeoStats/denue/denue_00_62_csv/conjunto_de_datos/denue_inegi_62_.csv'

    # Ruta de archivo parques DENUE
    file_path_parques = '/home/jay/repos/GeoStats/denue/denue_00_71_csv/conjunto_de_datos/denue_inegi_71_.csv'

    # Ruta de archivo mayoreo DENUE
    file_path_mayoreo = '/home/jay/repos/GeoStats/denue/denue_00_43_csv/conjunto_de_datos/denue_inegi_43_.csv'

## Dado una lista de terminos en una lista de columnas se filtra.

In [4]:
columnas_para_filtrar = ['nombre_act', 'raz_social', 'nom_estab']

In [5]:
# Filtramos escuelas 
df_escuelas = filtrar_dataframe(file_path_escuelas, columnas_para_filtrar, [])

In [ ]:
# Filtramos parques
terminos_parques = ['parques', 'jardin', 'plaza', 'bosque']
df_parques = filtrar_dataframe(file_path_parques, columnas_para_filtrar, terminos_parques)

In [ ]:
# Salud es complicado, porque queremos hospitales, publicas y privadas separadas
terminos_privadas = ['consultorio', 'privado']
terminos_publicas = ['imss', 'publico', 'isste']
terminos_hospitales = ['hospital', 'cirugia']

df_salud_privadas = filtrar_dataframe(file_path_salud, columnas_para_filtrar, terminos_privadas)
df_salud_publicas = filtrar_dataframe(file_path_salud, columnas_para_filtrar, terminos_publicas)
df_salud_hospitales = filtrar_dataframe(file_path_salud, columnas_para_filtrar, terminos_hospitales)

In [6]:
# Para los negocios de alcohol tenemos que unir todos los dataframes al final
terminos_alcohol = ['alcohol', 'licor', 'cerveza', 'bar', 'cantina', 'vinos', 'bebidas alcoholicas']

df_alcohol_list = []

for path in paths_negocios_denue:
    df_filtrado = filtrar_dataframe(path, columnas_para_filtrar, terminos_alcohol)
    df_alcohol_list.append(df_filtrado)

df_alcohol = pd.concat(df_alcohol_list, ignore_index=True)

    

In [ ]:
# Fabricas
terminos_fabricas = ['fabrica', 'fabricacion', 'industria', 'industrias', 'industrial', 'industriales']
df_fabricas = filtrar_dataframe(file_path_mayoreo, columnas_para_filtrar, terminos_fabricas)

# EDA

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(10, 6))

# Plot each dataframe with a different color
plt.scatter(df_parques['longitud'], df_parques['latitud'], color='green', label='parques')
plt.scatter(df_escuelas['longitud'], df_escuelas['latitud'], color='blue', label='escuelas')
plt.scatter(df_salud_privadas['longitud'], df_salud_privadas['latitud'], color='red', label='salud privada')
plt.scatter(df_salud_publicas['longitud'], df_salud_publicas['latitud'], color='orange', label='salud publica')
plt.scatter(df_salud_hospitales['longitud'], df_salud_hospitales['latitud'], color='purple', label='hospitales')
plt.scatter(df_alcohol['longitud'], df_alcohol['latitud'], color='black', label='alcohol')
plt.scatter(df_fabricas['longitud'], df_fabricas['latitud'], color='yellow', label='fabricas')

# Add a title and labels
plt.title('Scatter Plot of Different Establishments')
plt.xlabel('Longitude')
plt.ylabel('Latitude')

# Add a legend to identify the datasets
plt.legend()

# Show the plot
plt.show()

# Menor distancia entre dos dataframes
## Se usa sklearn para el compute
## Se guarda el indice y la distancia del establecimiento mas cercano a la escuela

In [ ]:
# Los N locales mas cercanos
N = 5
distances, indices = find_nearest_n_places(df_escuelas, df_alcohol, n_neighbors=N)

# Guardar id y distancias
for i in range(N):
    df_escuelas[f'closest_alcohol_id_{i+1}'] = df_alcohol.iloc[indices[:, i]]['id'].values
    df_escuelas[f'closest_distance_km_{i+1}'] = distances[:, i]    


In [13]:
from scripts.distance import find_nearest_within_radius


R = 0.2  # For example, a 10 km radius
distances, indices = find_nearest_within_radius(df_escuelas, df_alcohol, radius=R)


In [10]:
df_escuelas['closest_alcohol_ids'] = [
    ', '.join(df_alcohol.iloc[ind]['id'].astype(str).values) for ind in indices
]

# Store the distances as a CSV in a single column
df_escuelas['closest_distances_km'] = [
    ', '.join(map(str, dist)) for dist in distances
]

df_escuelas['closest_alcohol_names'] = [
    ', '.join(df_alcohol.iloc[ind]['nom_estab'].values) for ind in indices
]

In [14]:
df_escuelas['closest_alcohol_names']

88883    carniceria y abarrotes cavazos, deposito yulia...
88884    abarrotes medina, abarrotes don chuy, abarrote...
88885                lecheria 44, snack martha, correccion
88886                                                     
88887                                                     
                               ...                        
95773                                                     
95774                                                     
95775                                              timbare
95776                                                     
95777                                  modelorama mar rojo
Name: closest_alcohol_names, Length: 6895, dtype: object